## Intall libs

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install kaggle

In [ ]:
# !apt-get install tree

In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.4/756.4 kB 8.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
# !mkdir /.kaggle
# !mv kaggle.json /.kaggle
# !mv /.kaggle /root/
# !chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download sainikhileshreddy/food-recognition-2022

!unzip food-recognition-2022.zip

!rm food-recognition-2022.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: raw_data/public_training_set_release_2.0/images/165494.jpg  
  inflating: raw_data/public_training_set_release_2.0/images/165503.jpg  
  inflating: raw_data/public_training_set_release_2.0/images/165507.jpg  
  inflating: raw_data/public_training_set_release_2.0/images/165508.jpg  
  inflating: raw_data/public_training_set_release_2.0/images/165509.jpg  
  inflating: raw_data/public_training_set_release_2.0/images/165510.jpg  
  inflating: raw_data/public_training_set_release_2.0/images/165515.jpg  
  inflating: raw_data/public_training_set_release_2.0/images/165519.jpg  
  inflating: raw_data/public_training_set_release_2.0/images/165522.jpg  
  inflating: raw_data/public_training_set_release_2.0/images/165529.jpg  
  inflating: raw_data/public_training_set_release_2.0/images/165536.jpg  
  inflating: raw_data/public_training_set_release_2.0/images/165543.jpg  
  inflating: raw_data/public_training_set_re

In [ ]:
# !tree -L 3 /content/

/content/
├── hub
│   ├── test
│   │   ├── dataset_lock.lock
│   │   ├── dataset_meta.json
│   │   ├── images
│   │   └── images_meta
│   ├── train
│   │   ├── areas
│   │   ├── boxes
│   │   ├── categories
│   │   ├── dataset_lock.lock
│   │   ├── dataset_meta.json
│   │   ├── images
│   │   ├── images_meta
│   │   ├── iscrowds
│   │   ├── masks
│   │   └── super_categories
│   └── val
│       ├── areas
│       ├── boxes
│       ├── categories
│       ├── dataset_lock.lock
│       ├── dataset_meta.json
│       ├── images
│       ├── images_meta
│       ├── iscrowds
│       ├── masks
│       └── super_categories
├── raw_data
│   ├── public_test_release_2.0
│   │   └── images
│   ├── public_training_set_release_2.0
│   │   ├── annotations.json
│   │   └── images
│   └── public_validation_set_2.0
│       ├── annotations.json
│       └── images
├── sample_data
│   ├── anscombe.json
│   ├── california_housing_test.csv
│   ├── california_housing_train.csv
│   ├── mnist_test.csv
│   ├── mnis

## Importing Libs

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split
import torchvision.transforms as tt
import numpy as np
import pandas as pd
import os
from PIL import Image
import json
import albumentations as A
from albumentations.pytorch import ToTensorV2

## Data Preperation for YOLO

In [ ]:
def load_json_to_dict(json_path):
    with open(json_path, 'r') as f:
        return json.load(f)

In [ ]:
train_json = load_json_to_dict('raw_data/public_training_set_release_2.0/annotations.json')
val_json = load_json_to_dict('raw_data/public_validation_set_2.0/annotations.json')

In [ ]:
def check_images(dir, data):
  corrupted = []
  images = data['images']
  for image in images:
    path = os.path.join(dir, image['file_name'])
    try:
      img = Image.open(path)
      img.verify()
    except:
      print(f"Corrupted image: {path}")
      corrupted.append(image)
  return corrupted

def remove_corrupted(dir, data):
  num = 0
  for image in data:
    path = os.path.join(dir, image['file_name'])
    try:
      os.remove(path)
      num += 1
    except:
      print(f"Failed to remove corrupted image: {path}")

  return num

corrupted_imgs = check_images('raw_data/public_training_set_release_2.0/images', train_json)
num = remove_corrupted('raw_data/public_training_set_release_2.0/images', corrupted_imgs)
print(f"Removed {num} corrupted images")

Removed 0 corrupted images


In [ ]:
def label_prep(dir, data):
  classes = {cat['id']: idx for idx, cat in enumerate(train_json['categories'])}

  seen_anns = set()
  anns = {}
  for ann in data['annotations']:
    image_id = ann['image_id']
    if image_id not in anns:
      anns[image_id] = []
    unique_id = (image_id, ann['category_id'], tuple(ann['bbox']))
    if unique_id in seen_anns:
      continue

    seen_anns.add(unique_id)
    anns[image_id].append({
        'id': classes[ann['category_id']],
        'bbox': ann['bbox']
    })

  for image in data['images']:
    width, height = image['width'], image['height']

    image_id = image['id']
    # padding
    if len(str(image_id)) < 6:
      image_id = '0' * (6 - len(str(image_id))) + str(image_id)

    path = os.path.join(dir, f'{image_id}.txt')

    with open(path, 'w') as f:
      for ann in anns[image['id']]:
        bbox = ann['bbox']
        line = f"{ann['id']} {(bbox[0] + bbox[2]/2)/width} {(bbox[1] + bbox[3]/2)/height} {bbox[2]/width} {bbox[3]/height}"
        f.write(line + '\n')

In [ ]:
!mkdir raw_data/public_training_set_release_2.0/labels
!mkdir raw_data/public_validation_set_2.0/labels

In [ ]:
label_prep('raw_data/public_training_set_release_2.0/labels', train_json)
label_prep('raw_data/public_validation_set_2.0/labels', val_json)

In [ ]:
classes = [cat['name'] for cat in train_json['categories']]

In [ ]:
def open_yaml(path):
  with open(path, 'w') as data_yaml:
    content =f"""
path: /content/raw_data
train: public_training_set_release_2.0/images
val: public_validation_set_2.0/images
test: public_test_release_2.0/images
nc: 498
names: {[cat for cat in classes]}
    """
    data_yaml.write(content)
    data_yaml.close()

open_yaml('data.yaml')

# Fine-tuning YOLOv8

## Training (1st)

In [ ]:
from ultralytics import YOLO

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = YOLO('yolov8n.pt')
model.train(data='data.yaml', epochs=20, device=device, save=True)

Ultralytics YOLOv8.2.8 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=

100%|██████████| 755k/755k [00:00<00:00, 41.2MB/s]


Overriding model.yaml nc=80 with nc=498

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralyti

train: Scanning /content/raw_data/public_training_set_release_2.0/labels... 39962 images, 0 backgrounds, 190 corrupt: 100%|██████████| 39962/39962 [01:08<00:00, 580.45it/s]

train: WARNING ⚠️ /content/raw_data/public_training_set_release_2.0/images/006519.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0583       1.065]
train: WARNING ⚠️ /content/raw_data/public_training_set_release_2.0/images/008618.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/raw_data/public_training_set_release_2.0/images/008619.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.1833]
train: WARNING ⚠️ /content/raw_data/public_training_set_release_2.0/images/008620.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/raw_data/public_training_set_release_2.0/images/008621.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/raw_data/public_training_set_release_2.0/images/008626.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/raw_data/public_training_set_release_2.0/images/008627.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/raw_data/public_

...

      20/20      4.64G      1.333      2.921       1.93         23        640: 100%|██████████| 2486/2486 [17:57<00:00,  2.31it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:15<00:00,  2.07it/s]


                   all        994       1812      0.609      0.167      0.182      0.123

20 epochs completed in 6.295 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 7.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 7.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.8 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
Model summary (fused): 168 layers, 3461318 parameters, 0 gradients, 10.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:17<00:00,  1.81it/s]


                   all        994       1812      0.609      0.168      0.182      0.123
       bread-wholemeal        994         36      0.164      0.605      0.211      0.135
                   jam        994         21      0.303      0.714      0.558      0.303
                 water        994         75      0.393      0.827      0.697      0.381
       bread-sourdough        994          3          0          0     0.0416     0.0224
                banana        994          7      0.239      0.857      0.676      0.452
           soft-cheese        994          6          0          0      0.012    0.00511
               ham-raw        994          6      0.208      0.441      0.136     0.0853
           hard-cheese        994         12      0.179      0.417      0.205      0.105
        cottage-cheese        994          4          1          0     0.0149    0.00854
      bread-half-white        994          1          0          0          0          0
  coffee-with-caffein

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  47,  48,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  76,  78,  80,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 109, 110, 111, 113, 114, 115, 116, 117, 118, 119, 120, 122, 123, 124, 125, 126, 128, 129, 130, 132, 133, 135, 136, 137, 138, 140,
       141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 157, 158, 159, 160, 161, 162, 163, 164, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 185, 186, 187, 188, 191, 192, 193, 194, 195, 196, 197, 198, 200, 201, 202, 203, 2

In [ ]:
results = model('raw_data/public_training_set_release_2.0/images/006335.jpg')
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen
    result.save(filename='result.jpg')  # save to dis


image 1/1 /content/raw_data/public_training_set_release_2.0/images/006335.jpg: 640x640 (no detections), 14.2ms
Speed: 3.1ms preprocess, 14.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


In [ ]:
results_dict = {
    'metrics/precision(B)': 0.709659800968203,
    'metrics/recall(B)': 0.08593485687335754,
    'metrics/mAP50(B)': 0.11885189650929667,
    'metrics/mAP50-95(B)': 0.0765313557939187,
    'fitness': 0.08076340986545649
}
for key, val in results_dict:

{'metrics/precision(B)': 0.709659800968203, 'metrics/recall(B)': 0.08593485687335754, 'metrics/mAP50(B)': 0.11885189650929667, 'metrics/mAP50-95(B)': 0.0765313557939187, 'fitness': 0.08076340986545649}


## Continue Training (2nd)

In [ ]:
model = YOLO('runs/detect/train/weights/last.pt')
model.train(data='data.yaml', epochs=20, device=device, save=True, optimizer='AdamW', lr0=1e-6)

Ultralytics YOLOv8.2.8 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
engine/trainer: task=detect, mode=train, model=runs/detect/train/weights/last.pt, data=data.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sh

train: Scanning /content/raw_data/public_training_set_release_2.0/labels.cache... 39962 images, 0 backgrounds, 190 corrupt: 100%|██████████| 39962/39962 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/raw_data/public_training_set_release_2.0/images/006519.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0583       1.065]
train: WARNING ⚠️ /content/raw_data/public_training_set_release_2.0/images/008618.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/raw_data/public_training_set_release_2.0/images/008619.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.1833]
train: WARNING ⚠️ /content/raw_data/public_training_set_release_2.0/images/008620.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/raw_data/public_training_set_release_2.0/images/008621.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/raw_data/public_training_set_release_2.0/images/008626.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/raw_data/public_training_set_release_2.0/images/008627.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/raw_data/public_

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/raw_data/public_validation_set_2.0/labels.cache... 1000 images, 0 backgrounds, 6 corrupt: 100%|██████████| 1000/1000 [00:00<?, ?it/s]

...

      20/20      6.93G      1.372      3.011      1.963         23        640: 100%|██████████| 2486/2486 [18:54<00:00,  2.19it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:15<00:00,  2.12it/s]


                   all        994       1812      0.566      0.165      0.163      0.104

20 epochs completed in 6.513 hours.
Optimizer stripped from runs/detect/train5/weights/last.pt, 7.2MB
Optimizer stripped from runs/detect/train5/weights/best.pt, 7.1MB

Validating runs/detect/train5/weights/best.pt...
Ultralytics YOLOv8.2.8 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
Model summary (fused): 168 layers, 3461318 parameters, 0 gradients, 10.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:18<00:00,  1.72it/s]


                   all        994       1812      0.581      0.172      0.172      0.109
       bread-wholemeal        994         36      0.156      0.556      0.213      0.128
                   jam        994         21      0.255      0.714      0.498      0.249
                 water        994         75      0.384       0.84      0.688      0.367
       bread-sourdough        994          3      0.414      0.333      0.182       0.11
                banana        994          7      0.257      0.857      0.706      0.463
           soft-cheese        994          6          0          0    0.00248    0.00132
               ham-raw        994          6      0.157      0.341      0.155      0.102
           hard-cheese        994         12      0.167       0.25       0.08     0.0367
        cottage-cheese        994          4          1          0     0.0182     0.0121
      bread-half-white        994          1          0          0          0          0
  coffee-with-caffein

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  47,  48,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  76,  78,  80,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 109, 110, 111, 113, 114, 115, 116, 117, 118, 119, 120, 122, 123, 124, 125, 126, 128, 129, 130, 132, 133, 135, 136, 137, 138, 140,
       141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 157, 158, 159, 160, 161, 162, 163, 164, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 185, 186, 187, 188, 191, 192, 193, 194, 195, 196, 197, 198, 200, 201, 202, 203, 2

## Continue Training (3rd)

In [ ]:
from ultralytics import YOLO

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = YOLO('last.pt')
model.train(data='data.yaml', epochs=10, device=device, save=True, optimizer='AdamW', lr0=1e-6)

Ultralytics YOLOv8.2.9 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
engine/trainer: task=detect, mode=train, model=last.pt, data=data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=Tr

100%|██████████| 755k/755k [00:00<00:00, 42.4MB/s]



                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

100%|██████████| 6.23M/6.23M [00:00<00:00, 172MB/s]


AMP: checks passed ✅


...

      10/10      4.66G      1.332      2.871      1.922         24        640: 100%|██████████| 2486/2486 [18:10<00:00,  2.28it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:16<00:00,  2.00it/s]


                   all        994       1812      0.579      0.174      0.183      0.123

10 epochs completed in 3.098 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 7.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 7.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.9 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
Model summary (fused): 168 layers, 3461318 parameters, 0 gradients, 10.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:18<00:00,  1.72it/s]


                   all        994       1812      0.583      0.174      0.186      0.124
       bread-wholemeal        994         36       0.16      0.583      0.254       0.16
                   jam        994         21      0.237      0.651      0.512      0.286
                 water        994         75      0.394      0.827      0.688      0.373
       bread-sourdough        994          3          0          0     0.0801     0.0484
                banana        994          7      0.288      0.857      0.675      0.437
           soft-cheese        994          6          0          0     0.0102    0.00469
               ham-raw        994          6      0.237        0.5      0.159     0.0926
           hard-cheese        994         12      0.127      0.333       0.18     0.0969
        cottage-cheese        994          4          1          0     0.0199     0.0122
      bread-half-white        994          1          0          0          0          0
  coffee-with-caffein

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  47,  48,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  76,  78,  80,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 109, 110, 111, 113, 114, 115, 116, 117, 118, 119, 120, 122, 123, 124, 125, 126, 128, 129, 130, 132, 133, 135, 136, 137, 138, 140,
       141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 157, 158, 159, 160, 161, 162, 163, 164, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 185, 186, 187, 188, 191, 192, 193, 194, 195, 196, 197, 198, 200, 201, 202, 203, 2

# Evaluation

In [ ]:
from ultralytics import YOLO

model = YOLO('best.pt')
metrics = model.val(data='data.yaml', save_json=True, iou=0.5, device='0', plots=True)

Ultralytics YOLOv8.2.12 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3461318 parameters, 0 gradients, 10.3 GFLOPs


val: Scanning /content/raw_data/public_validation_set_2.0/labels.cache... 1000 images, 0 backgrounds, 6 corrupt: 100%|██████████| 1000/1000 [00:00<?, ?it/s]

val: WARNING ⚠️ /content/raw_data/public_validation_set_2.0/images/126986.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.1375]
val: WARNING ⚠️ /content/raw_data/public_validation_set_2.0/images/130163.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0586]
val: WARNING ⚠️ /content/raw_data/public_validation_set_2.0/images/132133.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.3223      1.3284]
val: WARNING ⚠️ /content/raw_data/public_validation_set_2.0/images/132371.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.3604      1.3693]
val: WARNING ⚠️ /content/raw_data/public_validation_set_2.0/images/135191.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.3018]
val: WARNING ⚠️ /content/raw_data/public_validation_set_2.0/images/144957.jpg: ignoring corrupt image/label: non-normalized or out of bounds 


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 63/63 [00:19<00:00,  3.23it/s]


                   all        994       1812      0.577      0.175      0.176      0.109
       bread-wholemeal        994         36      0.168      0.528      0.219      0.125
                   jam        994         21      0.304      0.667      0.523      0.254
                 water        994         75      0.459      0.827      0.695      0.362
       bread-sourdough        994          3      0.388      0.333      0.183       0.11
                banana        994          7      0.326      0.857      0.725      0.448
           soft-cheese        994          6          0          0    0.00383    0.00192
               ham-raw        994          6      0.186      0.382      0.171      0.105
           hard-cheese        994         12      0.169       0.25     0.0915     0.0426
        cottage-cheese        994          4          1          0     0.0223     0.0147
      bread-half-white        994          1          0          0          0          0
  coffee-with-caffein

In [ ]:
print('mAP at IoU thresholds from 0.5 to 0.95: {:.3f}'.format(metrics.box.map))    # mAP at IoU thresholds from 0.5 to 0.95
print('mAP at IoU threshold of 0.5: {:.3f}'.format(metrics.box.map50))  # mAP at IoU threshold of 0.5
print('mAP at IoU threshold of 0.7: {:.3f}'.format(metrics.box.map75))  # mAP at IoU threshold of 0.75


mAP at IoU thresholds from 0.5 to 0.95: 0.109
mAP at IoU threshold of 0.5: 0.176
mAP at IoU threshold of 0.7: 0.117
